In [40]:
import pandas as pd
import numpy as np
import ast
from nltk.stem.porter import PorterStemmer
import re
import joblib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Loading Data

In [2]:
movies=pd.read_csv("tmdb_5000_movies.csv")
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [3]:
credits=pd.read_csv("tmdb_5000_credits.csv")
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
credits=credits.rename(columns={"movie_id":"id"})
credits.isna().sum()

id       0
title    0
cast     0
crew     0
dtype: int64

# Manupulating credit data

In [5]:
credits["cast"].iloc[100]

'[{"cast_id": 4, "character": "Daisy", "credit_id": "52fe43e2c3a36847f807632b", "gender": 1, "id": 112, "name": "Cate Blanchett", "order": 0}, {"cast_id": 5, "character": "Benjamin Button", "credit_id": "52fe43e2c3a36847f807632f", "gender": 2, "id": 287, "name": "Brad Pitt", "order": 1}, {"cast_id": 6, "character": "Elizabeth Abbott", "credit_id": "52fe43e2c3a36847f8076333", "gender": 1, "id": 3063, "name": "Tilda Swinton", "order": 2}, {"cast_id": 10, "character": "Caroline", "credit_id": "52fe43e2c3a36847f8076343", "gender": 1, "id": 15887, "name": "Julia Ormond", "order": 3}, {"cast_id": 7, "character": "Daisy Age 7", "credit_id": "52fe43e2c3a36847f8076337", "gender": 1, "id": 18050, "name": "Elle Fanning", "order": 4}, {"cast_id": 183, "character": "Daisy Age 10", "credit_id": "56ec883a9251414dba0022af", "gender": 1, "id": 219666, "name": "Madisen Beaty", "order": 5}, {"cast_id": 8, "character": "Thomas Button", "credit_id": "52fe43e2c3a36847f807633b", "gender": 2, "id": 973, "name

In [6]:
# Taking top 5 names

In [7]:
def get_top_5_names(x):
    L=[]
    content=ast.literal_eval(x)
    if(len(content)>=5):
        for i in range(0,5):
            name=content[i]["name"]
            name=name.replace(" ","")
            name=name.lower()
            L.append(name)
    else:
        for i in range(len(content)):
            name=content[i]["name"]
            name=name.replace(" ","")
            name=name.lower()
            L.append(name)
    return L
credits["cast"]=credits["cast"].apply(get_top_5_names)

In [8]:
credits["cast"][0]  # Perfect

['samworthington',
 'zoesaldana',
 'sigourneyweaver',
 'stephenlang',
 'michellerodriguez']

In [9]:
credits["crew"].iloc[10]  # Lets take director and producer out

'[{"credit_id": "553bef6a9251416874003c8f", "department": "Production", "gender": 2, "id": 3276, "job": "Casting", "name": "Roger Mussenden"}, {"credit_id": "553bee5592514135c800283e", "department": "Production", "gender": 2, "id": 3805, "job": "Producer", "name": "Jon Peters"}, {"credit_id": "52fe42f8c3a36847f8030bb1", "department": "Production", "gender": 0, "id": 13192, "job": "Production Manager", "name": "Diane Macke"}, {"credit_id": "553bee6692514157f0000b54", "department": "Production", "gender": 2, "id": 10953, "job": "Producer", "name": "Gilbert Adler"}, {"credit_id": "52fe42f8c3a36847f8030b6f", "department": "Directing", "gender": 2, "id": 9032, "job": "Director", "name": "Bryan Singer"}, {"credit_id": "52fe42f8c3a36847f8030b75", "department": "Production", "gender": 2, "id": 9032, "job": "Producer", "name": "Bryan Singer"}, {"credit_id": "553bee83c3a36820ec002201", "department": "Writing", "gender": 2, "id": 9032, "job": "Story", "name": "Bryan Singer"}, {"credit_id": "553be

In [10]:
def get_deirector_producer(x):
    L=[]
    content=ast.literal_eval(x)
    for row in range(len(content)):
        if content[row]["job"]=="Director":
            name=content[row]["name"]
            name=name.replace(" ","")
            name=name.lower()
            L.append(name)
        elif content[row]["job"]=="Producer":
            name=content[row]["name"]
            name=name.replace(" ","")
            name=name.lower()
            L.append(name)
    
    return list(set(L))
credits["crew"]=credits["crew"].apply(get_deirector_producer)

In [11]:
credits["crew"][10]

['gilbertadler', 'jonpeters', 'bryansinger']

In [12]:
credits.head()

,id,title,cast,crew
0,19995,Avatar,"[samworthington, zoesaldana, sigourneyweaver, ...","[jonlandau, jamescameron]"
1,285,Pirates of the Caribbean: At World's End,"[johnnydepp, orlandobloom, keiraknightley, ste...","[chadoman, ericmcleod, peterkohn, jerrybruckhe..."
2,206647,Spectre,"[danielcraig, christophwaltz, léaseydoux, ralp...","[barbarabroccoli, michaelg.wilson, sammendes]"
3,49026,The Dark Knight Rises,"[christianbale, michaelcaine, garyoldman, anne...","[charlesroven, emmathomas, christophernolan]"
4,49529,John Carter,"[taylorkitsch, lynncollins, samanthamorton, wi...","[jimmorris, lindseycollins, andrewstanton, col..."


# Movies data Manupulation

In [13]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [14]:
# Columns to keep
# genres,id,keywords,overview,tagline,title

In [15]:
movies=movies[['genres','keywords','overview','tagline','title']]

In [16]:
movies.info(),movies.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genres    4803 non-null   object
 1   keywords  4803 non-null   object
 2   overview  4800 non-null   object
 3   tagline   3959 non-null   object
 4   title     4803 non-null   object
dtypes: object(5)
memory usage: 187.7+ KB


(None,
 genres        0
 keywords      0
 overview      3
 tagline     844
 title         0
 dtype: int64)

In [17]:
movies["genres"][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [18]:
def genre(x):
    L=[]
    content=ast.literal_eval(x)
    for i in range(len(content)):
        name=content[i]["name"]
        name=name.replace(" ","")
        name=name.lower()
        L.append(name)
    return L
movies["genres"]=movies["genres"].apply(genre)

In [19]:
movies["genres"][0]

['action', 'adventure', 'fantasy', 'sciencefiction']

In [20]:
movies["keywords"][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [21]:
def keywords(x):
    L=[]
    content=ast.literal_eval(x)
    for i in range(len(content)):
        name=content[i]["name"]
        name=name.replace(" ","")
        name=name.lower()
        L.append(name)
    return L
movies["keywords"]=movies["keywords"].apply(keywords)

In [22]:
movies["keywords"][0]

['cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d']

In [23]:
movies["overview"][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [24]:
def overview(x):
    if x is not np.nan:
        cleaned = re.sub(r'[^\w\s]', '', x)
        cleaned = cleaned.lower()
        L=cleaned.split(" ")
    else:
        L=[]
    return L
movies["overview"]=movies["overview"].apply(overview)

In [25]:
movies["overview"][0]

['in',
 'the',
 '22nd',
 'century',
 'a',
 'paraplegic',
 'marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'pandora',
 'on',
 'a',
 'unique',
 'mission',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization']

In [26]:
movies["tagline"][0]

'Enter the World of Pandora.'

In [27]:
def overview(x):
    if x is not np.nan:
        cleaned = re.sub(r'[^\w\s]', '', x)
        cleaned = cleaned.lower()
        L=cleaned.split(" ")
    else:
        L=[]
    return L
movies["tagline"]=movies["tagline"].apply(overview)

In [28]:
movies["tagline"][0]

['enter', 'the', 'world', 'of', 'pandora']

# Combining both

In [29]:
df=pd.merge(movies, credits, on="title")

In [30]:
df.head()

,genres,keywords,overview,tagline,title,id,cast,crew
0,"[action, adventure, fantasy, sciencefiction]","[cultureclash, future, spacewar, spacecolony, ...","[in, the, 22nd, century, a, paraplegic, marine...","[enter, the, world, of, pandora]",Avatar,19995,"[samworthington, zoesaldana, sigourneyweaver, ...","[jonlandau, jamescameron]"
1,"[adventure, fantasy, action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[captain, barbossa, long, believed, to, be, de...","[at, the, end, of, the, world, the, adventure,...",Pirates of the Caribbean: At World's End,285,"[johnnydepp, orlandobloom, keiraknightley, ste...","[chadoman, ericmcleod, peterkohn, jerrybruckhe..."
2,"[action, adventure, crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[a, cryptic, message, from, bonds, past, sends...","[a, plan, no, one, escapes]",Spectre,206647,"[danielcraig, christophwaltz, léaseydoux, ralp...","[barbarabroccoli, michaelg.wilson, sammendes]"
3,"[action, crime, drama, thriller]","[dccomics, crimefighter, terrorist, secretiden...","[following, the, death, of, district, attorney...","[the, legend, ends]",The Dark Knight Rises,49026,"[christianbale, michaelcaine, garyoldman, anne...","[charlesroven, emmathomas, christophernolan]"
4,"[action, adventure, sciencefiction]","[basedonnovel, mars, medallion, spacetravel, p...","[john, carter, is, a, warweary, former, milita...","[lost, in, our, world, found, in, another]",John Carter,49529,"[taylorkitsch, lynncollins, samanthamorton, wi...","[jimmorris, lindseycollins, andrewstanton, col..."


In [31]:
df["tags"]=df["genres"]+df["keywords"]+df["overview"]+df["tagline"]+df["cast"]+df["crew"]
df["tags"]=df["tags"].apply(lambda x: " ".join(x))

In [32]:
def stem(x):
    y=[]
    for string in x.split():
        y.append(PorterStemmer().stem(string))
    return " ".join(y)
df["tags"]=df["tags"].apply(stem)

# Vectorizing

In [33]:
vectorizer=TfidfVectorizer(stop_words="english",max_features=10000)
vectors=vectorizer.fit_transform(df["tags"]).toarray()

In [34]:
vectors.shape

(4809, 10000)

# Making similarity matrix

In [35]:
similarity_matrix=cosine_similarity(vectors)

In [36]:
similarity_matrix.shape

(4809, 4809)

In [37]:
def recommend(similarity_matrix):
    index=df[df["title"]=="Batman Begins"].index
    sim_scores = similarity_matrix[index].ravel()
    top_5=top5_idx = np.argsort(sim_scores)[-6:][::-1]
    top_5=top_5[1:]
    values=df.iloc[top_5]["title"].values
    print(values)

In [38]:
recommend(similarity_matrix)

['The Dark Knight' 'The Dark Knight Rises' 'Batman'
 'Batman v Superman: Dawn of Justice' 'Batman']


In [43]:
joblib.dump(df["title"],"movies.pkl","wb")

TypeError: '<' not supported between instances of 'str' and 'int'